In [3]:
from yahoo_fin import stock_info as si
import yfinance as yf
import pandas as pd
import os.path
import matplotlib.pyplot as plt
import seaborn as sns

class price:
    def __init__(self, ticker, time='max'):
        self.stock = yf.Ticker(ticker)
        self.info = self.stock.info
        #period : 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        self.price = self.stock.history(period=time)

class fundamental(price):
    def __init__(self, ticker, time='max', interval = 'year', level_of_rounding = 'million'):
        try:
            super().__init__(ticker, time)
            print("Please Wait...")
            
            if level_of_rounding == 'million':
                rounding = 1000000
            elif level_of_rounding == 'billion':
                rounding = 1000000000
            elif level_of_rounding == 'one':
                rounding = 1
            
            if interval == 'year':
                year_statement = True
            elif interval == 'quarter':
                year_statement = False
                
            #get data from Yahoo Finance
            self.balance_sheet_raw = si.get_balance_sheet(ticker,year_statement)
            self.income_statement_raw = si.get_income_statement(ticker, year_statement)
            self.cash_flow_raw = si.get_cash_flow(ticker, year_statement)
#             self.valuation = si.get_stats_valuation(ticker)
            #reshaping and pivoting
            self.total_share = int(self.info['marketCap']/self.price.iloc[-1,3])/rounding
            balance_sheet = self.balance_sheet_raw.loc[['totalAssets', 'totalStockholderEquity', 'retainedEarnings', 'totalLiab']]/rounding
            balance_sheet = balance_sheet.reset_index()
#             self.balance_sheet = balance_sheet.rename(columns={'Breakdown': 'Keys'})
            income_statement = self.income_statement_raw.loc[['totalRevenue', 'operatingIncome', 'netIncome']]/rounding
            income_statement = income_statement.reset_index()
#             self.income_statement = income_statement.rename(columns={'Breakdown': 'Keys'})
            cash_flow = self.cash_flow_raw.loc[['totalCashflowsFromInvestingActivities', 'totalCashFromFinancingActivities', 'totalCashFromOperatingActivities']]/rounding
            cash_flow = cash_flow.reset_index()
#             self.cash_flow = cash_flow.rename(columns={'Breakdown': 'Keys'})
            self.financial = self.balance_sheet.append([self.income_statement, self.cash_flow]).set_index('Keys').T
            #add ratio
            self.financial['debtEquityRatio'] = self.financial['totalLiab']/self.financial['totalStockholderEquity']
            self.financial['retainedEarningsEquityRatio'] = self.financial['retainedEarnings']/self.financial['totalStockholderEquity']
            self.financial['returnOnEquity'] = self.financial['netIncome']/self.financial['totalStockholderEquity']
            self.financial['operationalProfitMargin'] = self.financial['operatingIncome']/self.financial['totalRevenue']
            self.financial['netProfitMargin'] = self.financial['netIncome']/self.financial['totalRevenue']
            self.financial['earningPerShare'] = self.financial['netIncome']/self.total_share
            self.financial['bookValuePerShare'] = self.financial['totalStockholderEquity']/self.total_share
            print("Done")
        except:
            print('Error')
            pass

def write_csv(filename, df):
    if os.path.isfile(filename):
        df.to_csv(filename, header=False, index=False, mode='a')
    else:    
        df.to_csv(filename, header=True, index=False, mode='w')

def add_data(file, dataframe, ticker):
    dataframe['stockCode'] = ticker
    stock_columns = dataframe.columns.values.tolist()
    list_columns = [stock_columns[-1]] + stock_columns[0:-1]
    dataframe = dataframe.reindex(columns= list_columns)
    write_csv(file, dataframe)
                     
def add_stock_data(dataframe, fundamental_file, price_file, time = '5y', interval = 'year', no = 1):
    number = no - 1           
    for stock_ticker in dataframe['Kode'].iloc[number:]:
        print(number+1, 'Downloading Data :', stock_ticker)
        number += 1
        stock_data = fundamental(stock_ticker + '.JK', time, interval)
        #fundamental and price
        stock_value = stock_data.financial.T.reset_index()
        stock_price = stock_data.price.reset_index()[['Date', 'Close']]
#         stock_profile = stock_data.info.reset_index()[['Date', 'Close']]
        add_data(fundamental_file, stock_value, stock_ticker)
        add_data(price_file, stock_price, stock_ticker)
#         add_data(profile_file, stock_profile, stock_ticker)
        
def financial_plot(data_stock, ticker, time_interval, x_label, save_as):
    index_list = [4, 3, 2, 1]
    stock = data_stock.loc[data_stock['stockCode'] == ticker]
    stock_ticker = yf.Ticker(ticker+'.JK')
    stock_info = stock_ticker.info
    rounding = 1000000

    stock = stock.drop(columns = 'stockCode').T
    stock.columns = stock.iloc[0]
    stock = stock.drop('Keys')
    stock = stock.set_index([pd.Index(index_list)])
    stock = stock.iloc[::-1]
    stock = stock.reset_index()
    stock = stock.rename(columns = {'index': time_interval})

    sns.set(style="darkgrid")
    sns.set_color_codes("pastel")

    fig = plt.figure(figsize=(10,14))
    fig.suptitle(stock_info['longName']+' ('+ticker+')', fontsize=16)

    plt.subplot2grid((5, 2), (0, 0), colspan=2)
    plt.bar(stock[time_interval]+0.3, stock['totalAssets']/rounding, width=0.2, color='b', label='Asset')
    plt.bar(stock[time_interval]+0.1, stock['totalStockholderEquity']/rounding, width=0.2, color='g', label='Equity')
    plt.bar(stock[time_interval]-0.1, stock['retainedEarnings']/rounding, width=0.2, color='r', label='Retained Earning')
    plt.bar(stock[time_interval]-0.3, stock['totalLiab']/rounding, width=0.2, color='y', label='Liabilities')
    plt.xticks(index_list, x_label)
    plt.ylabel('Value (Trillion)')
    plt.title('Balance Sheet')
    plt.legend(bbox_to_anchor=(1, 1), loc = 'upper left')

    plt.subplot2grid((5, 2), (1, 0), colspan=2)
    plt.bar(stock[time_interval]-0.2, stock['totalRevenue']/rounding, width=0.2, color='b', label='Revenue')
    plt.bar(stock[time_interval], stock['operatingIncome']/rounding, width=0.2, color='g', label='Operating Income')
    plt.bar(stock[time_interval]+0.2, stock['netIncome']/rounding, width=0.2, color='r', label='Net Income')
    plt.xticks(index_list, x_label)
    plt.ylabel('Value (Trillion)')
    plt.title('Income Statement')
    plt.legend(bbox_to_anchor=(1, 1), loc = 'upper left')

    plt.subplot2grid((5, 2), (2, 0), colspan=2)
    plt.bar(stock[time_interval]-0.2, stock['totalCashflowsFromInvestingActivities']/rounding, width=0.2, color='b', label='Investing')
    plt.bar(stock[time_interval], stock['totalCashFromFinancingActivities']/rounding, width=0.2, color='g', label='Financing')
    plt.bar(stock[time_interval]+0.2, stock['totalCashFromOperatingActivities']/rounding, width=0.2, color='r', label='Operating')
    plt.xticks(index_list,  x_label)
    plt.ylabel('Value (Trillion)')
    plt.title('Cash Flow')
    plt.legend(bbox_to_anchor=(1, 1), loc = 'upper left')

    plt.subplot2grid((5, 2), (3, 0))
    plt.bar(stock[time_interval], stock['earningPerShare'], width=0.2, color='b', label='EPS')
    plt.xticks(index_list, x_label)
    plt.legend(bbox_to_anchor=(-0.12, 1), loc='upper right')

    plt.subplot2grid((5, 2), (3, 1))
    plt.bar(stock[time_interval], stock['bookValuePerShare'], width=0.2, color='g', label='BVPS')
    plt.xticks(index_list, x_label)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.subplot2grid((5, 2), (4, 0))
    plt.bar(stock[time_interval]-0.2, stock['debtEquityRatio'], width=0.2, color='b', label='DER')
    plt.bar(stock[time_interval], stock['retainedEarningsEquityRatio'], width=0.2, color='g', label='RER')
    plt.xticks(index_list, x_label)
    plt.legend(bbox_to_anchor=(-0.12, 1), loc='upper right')

    plt.subplot2grid((5, 2), (4, 1))
    plt.bar(stock[time_interval]+0.2, stock['operationalProfitMargin'], width=0.2, color='b', label='OPM')
    plt.bar(stock[time_interval], stock['netProfitMargin'], width=0.2, color='g', label='NPM')
    plt.bar(stock[time_interval]-0.2, stock['returnOnEquity'], width=0.2, color='r', label='ROE')
    plt.xticks(index_list, x_label)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.tight_layout()
    plt.savefig(save_as)

In [20]:
#read excel
# jii70 = pd.read_excel('JII70.xlsx', engine = 'openpyxl', header=0)
jii70['Kode'].unique()

array(['AALI', 'ACES', 'ADHI', 'ADRO', 'AKRA', 'ANTM', 'BIRD', 'BMTR',
       'BRIS', 'BRPT', 'BTPS', 'CLEO', 'CPIN', 'CTRA', 'DMAS', 'ERAA',
       'EXCL', 'HRUM', 'ICBP', 'INAF', 'INCO', 'INDF', 'INKP', 'INTP',
       'IPTV', 'ISAT', 'ITMG', 'JPFA', 'JRPT', 'KAEF', 'KINO', 'KLBF',
       'LINK', 'LPPF', 'LSIP', 'MAPI', 'MDKA', 'MIKA', 'MNCN', 'MTDL',
       'MYOR', 'PGAS', 'POWR', 'PTBA', 'PTPP', 'PWON', 'RALS', 'ROTI',
       'SCMA', 'SIDO', 'SILO', 'SIMP', 'SMBR', 'SMGR', 'SMRA', 'SMSM',
       'SSIA', 'TINS', 'TKIM', 'TLKM', 'TPIA', 'UCID', 'ULTJ', 'UNTR',
       'UNVR', 'WEGE', 'WIKA', 'WOOD', 'WSBP', 'WTON'], dtype=object)

In [3]:
add_stock_data(jii70, 'JII70_fundamental_year.csv', 'JII70_price_year.csv', time='5y', interval='year')

In [46]:
# df = price('AALI.JK', '1d')
# dataframe = pd.DataFrame(df.info)
stock_profile = df.info['sector']
stock_profile

'Consumer Defensive'

In [17]:
import datetime

data_price = pd.read_csv('JII70_price_quarter.csv')
# price.loc[:,'Date'] = price['Date'].astype('datetime64[ns]')
# price1 = price[price['Date'].dt.year == '2016']
# price = data_price.loc[data_price['stockCode'] == 'ULTJ']
# price1['per'] = price['Close']
# price1['pbv'] = price['Close']
data_price['stockCode'].unique()

array(['AALI', 'ACES', 'ADHI', 'ADRO', 'AKRA', 'ANTM', 'BIRD', 'BMTR',
       'BRIS', 'BRPT', 'BTPS', 'CLEO', 'CPIN', 'CTRA', 'DMAS', 'ERAA',
       'EXCL', 'HRUM', 'ICBP', 'INAF', 'INCO', 'INDF', 'INKP', 'INTP',
       'IPTV', 'ISAT', 'ITMG', 'JPFA', 'JRPT', 'KAEF', 'KINO', 'KLBF',
       'LINK', 'LPPF', 'LSIP', 'MAPI', 'MDKA', 'MIKA', 'MNCN', 'MTDL',
       'MYOR', 'PGAS', 'POWR', 'PTBA', 'PTPP', 'PWON', 'RALS', 'ROTI',
       'SCMA', 'SIDO', 'SILO', 'SIMP', 'SMBR', 'SMGR', 'SMRA', 'SMSM',
       'SSIA', 'TINS', 'TKIM', 'TLKM', 'TPIA', 'UCID', 'ULTJ', 'UNTR',
       'UNVR', 'WEGE', 'WIKA', 'WOOD', 'WSBP', 'WTON'], dtype=object)

In [2]:
quarter = pd.read_csv('JII70_fundamental_quarter.csv')
yearly = pd.read_csv('JII70_fundamental.csv')

q_interval = 'Quarter'
y_interval = 'Year'
quarter_label = ['20 Q3', '20 Q2', '20 Q1', '19 Q4']
year_label = [2019, 2018, 2017, 2016]

for stock_name in quarter['stockCode'].unique():
    print('Processing: ', stock_name)
    file_save = stock_name+'_quarter.png'
    financial_plot(quarter, stock_name, q_interval, quarter_label, file_save)
    plt.close()
    print('Done')

Processing:  AALI
Done
Processing:  ACES
Done
Processing:  ADHI
Done
Processing:  ADRO
Done
Processing:  AKRA
Done
Processing:  ANTM
Done
Processing:  BIRD
Done
Processing:  BMTR
Done
Processing:  BRIS
Done
Processing:  BRPT
Done
Processing:  BTPS
Done
Processing:  CLEO
Done
Processing:  CPIN
Done
Processing:  CTRA
Done
Processing:  DMAS
Done
Processing:  ERAA
Done
Processing:  EXCL
Done
Processing:  HRUM
Done
Processing:  ICBP
Done
Processing:  INAF
Done
Processing:  INCO
Done
Processing:  INDF
Done
Processing:  INKP
Done
Processing:  INTP
Done
Processing:  IPTV
Done
Processing:  ISAT
Done
Processing:  ITMG
Done
Processing:  JPFA
Done
Processing:  JRPT
Done
Processing:  KAEF
Done
Processing:  KINO
Done
Processing:  KLBF
Done
Processing:  LINK
Done
Processing:  LPPF
Done
Processing:  LSIP
Done
Processing:  MAPI
Done
Processing:  MDKA
Done
Processing:  MIKA
Done
Processing:  MNCN
Done
Processing:  MTDL
Done
Processing:  MYOR
Done
Processing:  PGAS
Done
Processing:  POWR
Done
Processing:

stockCode              1186
Keys                   1186
2019-12-31 00:00:00    1186
2018-12-31 00:00:00    1186
2017-12-31 00:00:00    1186
2016-12-31 00:00:00    1186
dtype: int64